# Group E: Assignment 3 Report

## Software Overview

<div style="text-align: justify">
DIgSILENT stands for "__DI__gital __SI__mu__L__ation of __E__lectrical __N__e__T__works", a software and consulting company providing engineering services in the field of electrical power systems for transmission, distribution, generation and industrial plants. PowerFactory is one of their software product which has ability to analyze transmission, distribution, and industrial electrical power systems. The main objective of this tool is to help planning and operation optimisation. In this exercise we are working with Medium Voltage distribution system structure  to implement configuration checking, planning, and optimising.
</div>


<div style="text-align: justify">
PowerFactory have API that can have interface with python script by using “powerfactory” module. This solution enables a Python script to have access to a comprehensive range of data available in PowerFactory, including all objects, attributes, commands, and functions. To gain access to the PowerFactory environment the command GetApplication() must be added.

</div>

## Requirement and Setup

<div style="text-align: justify">
In this assignment we need ImageMagick, free software to create, edit, and compose bitmap images. This feature used to create grid and geolocation visualization.
</div>

<div style="text-align: justify">
From PowerFactory objects required are:
</div>
1. Load flow calculation settings stored in Class _ComLDf_.
2. Study cases have a study time which defines the point in time the user wishes to analyse.
3. Object line element (_ElmLne_), transformer substation (_ElmTrfstat_), and feeder (_ElmFeeder_). A feeder is defined at a line or transformer end, and then the feeder definition algorithm searches the system from the definition point to determine the extent of the feeder.

<div style="text-align: justify">
From Python we need several libraries:
</div>
1. pandas, a data structures and data analysis tools
2. datetime and time, classes to manipulate date and time
3. os, miscellaneous operating system interface
4. PythonMagick, Python script binding to ImageMagick
5. Image from IPython.display, to display image in IPython Notebook

## Radiality Visual Check

<div style="text-align: justify">
Radial network structure have characteristic of no closed loop and unique path among any pair nodes. To do radiality visual checking, we build incidence matrix to evaluate visually whether the network is radially operated or not.
</div>

<img src="files/graphics/example.bmp" align="left"alt="Example geographical diagram" title="Example geographical diagram" />

<div style="text-align: justify">
From the supplied example configuration, we are building the instance matrix manually. Branch number is the number of the corresponding node, which has the incoming connection. For example, 1 -> 2 means, 1 connects to 2, so the branch number is 2. Size of the matrix is 1 + 5 + 17 = 23, 1 stands for the root 63 kV station, 5 for 20 kV substations, 17 for the trafo substations.
</div>

<div style="text-align: justify">
Here, we are configuring matrix as follows: 
</div>
1. First column is for the root node.
2. Next 5 columns are for the 20 kV substations.
3. Last 17 columns are for trafo substations.

<div style="text-align: justify">
Example: when we are refering the first trafo substation, corresponding index is calculated as follows:
1(root) + 5(20 kV stations) + 1(index we want) - 1(indexing starts from 0)
</div>

<div style="text-align: justify">
We can understand whether this network is radial or not by checking the rows of this matrix. If there are 2 or more 1s in a row, it means that there are 2 source nodes for the corresponding node. From this, we conclude that there are multiple paths to the same node, so the radiality is violated. Also if there is a closed loop, there has to be a 2 source nodes for a node. Only one exception to this is i.e 1(root)->2->3->1 , root node has an incoming node. In our case, this is impossible. So, whenever there is a closed loop in the network, there are at least 2 incoming nodes for one node.
</div>

<div style="text-align: justify">
As an algorithm, we may find the row sums and if the sum is larger than 0 (1 for source node, -1 for destination node) then there are 2 incoming nodes for that node. So the maximum value for the row sums should be 0.
</div>

<div style="text-align: justify">
However, to understand if the the case scenario a closed loop or not, one has to investigate further, since this algorithm only checks if it is radial or not, does not give a clue about closed loops.
</div>

## Optimal Configuration

<div style="text-align: justify">
With the help of Python script we can easily extract optimal radial configuration The objective function is to minimize power losses, for the actual study time. We define possible scenarios as shown in below configuration diagrams.
</div>


<div style="text-align: justify">
The first scenario implies switching off all redundant lines R(1),...,R(8). Hence, 
`first_scenario = ['R(1)', 'R(2)', 'R(3)', 'R(4)', 'R(5)', 'R(6)', 'R(7)','R(8)']`

We used it as a basis for the rest of scenarios. When building different scenarios, we have to make sure that:
* Network remains radial
* All transformer stations are fed
Hence we tried to look at the grid and think which lines are substitutable without the loss of network radiality and leaving any of he substations disconneted from power. E.g., in the second scenario we can turn off Line(9) but leave R(4). After experimenting, we came out with five different scenarios, including the first one:
</div>


```py
scenarios_ex4['firstScenario']  = ['R(1)', 'R(2)', 'R(3)', 'R(4)', 'R(5)', 'R(6)', 'R(7)','R(8)']
scenarios_ex4['secondScenario'] = ['R(1)', 'R(2)', 'R(3)', 'Line(9)', 'R(5)', 'R(6)', 'R(7)','R(8)']
scenarios_ex4['thirdScenario']  = ['R(1)', 'R(2)', 'R(3)', 'R(4)', 'Line(20)', 'R(6)', 'R(7)','R(8)']
scenarios_ex4['fourthScenario'] = ['R(1)', 'Line(15)', 'R(3)', 'R(4)', 'R(5)', 'R(6)', 'R(7)','R(8)']
scenarios_ex4['fifthScenario']  = ['R(1)', 'Line(15)', 'Line(35)', 'R(4)', 'R(5)', 'R(6)', 'R(7)','R(8)']
```

Which yielded the following results (key of the OrderedDict - name of the configuration (see above), value - Total Network Loss:

```py 
OrderedDict([('firstScenario', 4.260876644839072),
             ('thirdScenario', 4.48409292608085),
             ('fourthScenario', 4.510746161018621),
             ('fifthScenario', 4.535880279147257),
             ('secondScenario', 4.572679417943618)])
```

<div style="text-align: justify">
So far we achieved that all scenarios performed worse than a default one (first). Then we modify the next scenario - third to analyse. We leave R(5) swithed on but instead turn off Line(20). In fact, this made a path to trafoSubstation 14 longer than it was through Line(20).
</div>

<div style="text-align: justify">
Then we disable Line(32) and enable R(8) - network remains radial and all substations are energized. Thus, we energize trafoSubstations(26), (14), (11) and (28) directly from the power source through 20kV substation(5). 
</div>



```py
scenarios_ex4['sixthScenario']  = ['R(1)', 'R(2)', 'R(3)', 'R(4)', 'Line(20)', 'R(6)', 'R(7)','Line(32)']
```

Then finally we achieve our best scenario which performed better than a default scenario which shown in below diagrams.
</div>
<CENTER>
    <img  src="graphics\Exercise\Example\exercise4_best_configuration-2018-05-28 21.13.34-grid.png"/>
<CENTER>

<CENTER>
    <img  src="graphics\Exercise\Example\exercise4_best_configuration-2018-05-28 21.13.34.png"/>
<CENTER>

The sorted table of different configurations and its total power losses is presented below, ordered by the best performed scenario:

Configuration | Radial/All stations fed | Total Network Loss
:--- | :---: | :---:
`['R(1)', 'R(2)', 'R(3)', 'R(4)', 'Line(20)', 'R(6)', 'R(7)','Line(32)']` | True, True | 4.088599154242873
`['R(1)', 'R(2)', 'R(3)', 'R(4)', 'R(5)', 'R(6)', 'R(7)','R(8)']` | True, True | 4.260876644839072
`['R(1)', 'R(2)', 'R(3)', 'Line(9)', 'R(5)', 'R(6)', 'R(7)','R(8)']` | True, True | 4.48409292608085
`['R(1)', 'R(2)', 'R(3)', 'R(4)', 'Line(20)', 'R(6)', 'R(7)','R(8)']` | True, True | 4.510746161018621
`['R(1)', 'Line(15)', 'R(3)', 'R(4)', 'R(5)', 'R(6)', 'R(7)','R(8)']` | True, True | 4.535880279147257
`['R(1)', 'Line(15)', 'Line(35)', 'R(4)', 'R(5)', 'R(6)', 'R(7)','R(8)']` | True, True | 4.572679417943618

## Configuration Plan

To minimize total losses, we run the project for 24 hours that changes configuration on every hour. Here is an example scenario grid graphics for 4 AM 01.01.2015:

<CENTER>
    <img  src="graphics\Exercise\Example\exercise5_best_configuration_time_4-2018-05-28 00.52.10.png"/>
<CENTER>





Here is the <b>geographical</b> graphic for the same time period:


<CENTER>
    <img  src="graphics\Exercise\Example\exercise5_best_configuration_time_4-2018-05-28 00.52.10-geo.png"/>
<CENTER>

## Reconfiguration

<div style="text-align: justify">
In case of a fault event or an intentional outage of an element of the grid, circuit breakers on both side of the line will interrupt until the fault is resolved or planned outage is over. Meanwhile, in the shortest possible time, the grid must be restored that could feed all transformers. In this scenario we switch line(4) off until the end of this exercise. Line(4) connects 20 kV Substation(4) with Trafo Substations(9) and (17). Hence we need to find a way around, e.g., by not disabling R(8). We took configurations from exercise 4 (Optimal Configuration) as a basis and take into account length and load of lines. Below is the best configuration that we found:
</div>

<CENTER>
    <img  src="graphics\Exercise\Example\exercise6_best_configuration-2018-05-28 23.06.56-grid.png"/>
<CENTER>

<CENTER>
    <img  src="graphics\Exercise\Example\exercise6_best_configuration-2018-05-28 23.06.57.png"/>
<CENTER>

## Conclusion

As we have experienced, there are a number of possible scenarios even for a very static network. To try all different combinations even in this setting would take a huge amount of computation power. By optimizing it manually, we came up with different configurations and found success in one of them. However, after just one line break(Line 4 ), the network topology changed a lot. It is not very trivial to predict what would happen or what would be our solution in a case of very dynamic breaks of a combination of lines.

As human could be so slow for this task to accomplish, we need algorithms and/or some objective functions for this problem to be tackled in a larger scale.
